In [1]:
import pandas as pd
import numpy as np
import skfuzzy.control as ctrl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
🔷 D1 — Import Libraries & Load Membership Functions

In [3]:
import pandas as pd
import numpy as np
import skfuzzy.control as ctrl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# --- Import all universe variables and MFs from the external file ---
try:
    from fuzzy_memberships import (
        bmi_universe, age_universe, binary_universe, risk_universe,
        bmi_under, bmi_healthy, bmi_over, bmi_obese,
        age_young, age_middle, age_old,
        bp_normal, bp_high,
        smoke_no, smoke_yes,
        act_inactive, act_active,
        risk_vlow, risk_low, risk_med, risk_high, risk_vhigh
    )
    print("Fuzzy membership functions imported successfully from fuzzy_memberships.py.")
except ImportError:
    print("Error: Could not import from fuzzy_memberships.py. Ensure the file exists.")
    exit()

# --- Load Cleaned Data ---
try:
    df = pd.read_csv("../data/diabetes_clean.csv")
    print("Cleaned data loaded successfully.")
except FileNotFoundError:
    print("Error: 'diabetes_clean.csv' not found. Ensure the file path is correct.")
    exit()

Fuzzy membership functions imported successfully from fuzzy_memberships.py.
Cleaned data loaded successfully.


In [ ]:
🔷 D2 — Create Fuzzy Antecedents (Inputs) and Consequent (Output)

In [4]:
# 1. Antecedents (Inputs)
bmi = ctrl.Antecedent(bmi_universe, 'BMI')
age = ctrl.Antecedent(age_universe, 'Age')
highbp = ctrl.Antecedent(binary_universe, 'HighBP')
smoker = ctrl.Antecedent(binary_universe, 'Smoker')
physact = ctrl.Antecedent(binary_universe, 'PhysActivity')

# 2. Consequent (Output)
risk = ctrl.Consequent(risk_universe, 'Risk', defuzzify_method='centroid')

# 3. Assign the membership functions
bmi['under'] = bmi_under; bmi['healthy'] = bmi_healthy; bmi['over'] = bmi_over; bmi['obese'] = bmi_obese
age['young'] = age_young; age['middle'] = age_middle; age['old'] = age_old
highbp['normal'] = bp_normal; highbp['high'] = bp_high
smoker['no'] = smoke_no; smoker['yes'] = smoke_yes
physact['inactive'] = act_inactive; physact['active'] = act_active
risk['vlow'] = risk_vlow; risk['low'] = risk_low; risk['med'] = risk_med; risk['high'] = risk_high; risk['vhigh'] = risk_vhigh

In [ ]:
🔷 D3 — Write the Expanded Fuzzy Rule Base (20 Rules)

In [5]:
all_rules = []
# Inputs: bmi, age, highbp, smoker, physact
# Outputs: risk (vlow, low, med, high, vhigh)

# --- VLOW/LOW RULES (Baseline Health & Low Risk) ---
# 1. Optimal Health (VLow)
all_rules.append(ctrl.Rule(bmi['healthy'] & age['young'] & highbp['normal'] & smoker['no'] & physact['active'], risk['vlow']))
# 2. Underweight, but otherwise healthy (VLow)
all_rules.append(ctrl.Rule(bmi['under'] & highbp['normal'] & physact['active'] & smoker['no'], risk['vlow']))
# 3. Healthy, but slightly older (Low)
all_rules.append(ctrl.Rule(bmi['healthy'] & age['middle'] & highbp['normal'] & physact['active'], risk['low']))
# 4. Healthy, but smokes (Low)
all_rules.append(ctrl.Rule(bmi['healthy'] & age['young'] & highbp['normal'] & smoker['yes'], risk['low']))
# 5. Overweight, but young and active (Low)
all_rules.append(ctrl.Rule(bmi['over'] & age['young'] & highbp['normal'] & physact['active'], risk['low']))

# --- MEDIUM RULES (Mixed Risk: 1-2 major risk factors mitigated by others) ---
# 6. Overweight and inactive (Medium)
all_rules.append(ctrl.Rule(bmi['over'] & physact['inactive'] & highbp['normal'], risk['med']))
# 7. Obese, but young and active (Medium)
all_rules.append(ctrl.Rule(bmi['obese'] & age['young'] & physact['active'], risk['med']))
# 8. High BP, but young and non-smoker (Medium)
all_rules.append(ctrl.Rule(highbp['high'] & age['young'] & smoker['no'] & physact['active'], risk['med']))
# 9. Healthy, but old and inactive (Medium)
all_rules.append(ctrl.Rule(bmi['healthy'] & age['old'] & physact['inactive'], risk['med']))
# 10. Smoker and inactive, but normal weight (Medium)
all_rules.append(ctrl.Rule(smoker['yes'] & physact['inactive'] & highbp['normal'], risk['med']))
# 11. Overweight and Middle-aged, but healthy otherwise (Medium)
all_rules.append(ctrl.Rule(bmi['over'] & age['middle'] & highbp['normal'] & smoker['no'], risk['med']))
# 12. High BP and Middle-aged, but normal weight (Medium)
all_rules.append(ctrl.Rule(highbp['high'] & age['middle'] & bmi['healthy'], risk['med']))

# --- HIGH RULES (Serious Risk: Multiple moderate/major factors aligned) ---
# 13. Obese and middle-aged (High)
all_rules.append(ctrl.Rule(bmi['obese'] & age['middle'] & highbp['normal'], risk['high']))
# 14. High BP and smoker (High)
all_rules.append(ctrl.Rule(highbp['high'] & smoker['yes'] & age['middle'], risk['high']))
# 15. Overweight, old, and inactive (High)
all_rules.append(ctrl.Rule(bmi['over'] & age['old'] & physact['inactive'], risk['high']))
# 16. Obese and inactive (High)
all_rules.append(ctrl.Rule(bmi['obese'] & physact['inactive'], risk['high']))
# 17. High BP and old (High)
all_rules.append(ctrl.Rule(highbp['high'] & age['old'] & smoker['no'], risk['high']))

# --- VHIGH RULES (Critical Risk: Alignment of 2+ major factors) ---
# 18. Obese, High BP, and Old (VHigh - Triple Threat)
all_rules.append(ctrl.Rule(bmi['obese'] & highbp['high'] & age['old'], risk['vhigh']))
# 19. Obese, High BP, and Smoker (VHigh)
all_rules.append(ctrl.Rule(bmi['obese'] & highbp['high'] & smoker['yes'], risk['vhigh']))
# 20. Old, High BP, and Inactive (VHigh)
all_rules.append(ctrl.Rule(age['old'] & highbp['high'] & physact['inactive'], risk['vhigh']))

print(f"Total number of rules created: {len(all_rules)}")

Total number of rules created: 20


In [ ]:
D4 — Build the Fuzzy Control System

In [7]:

risk_ctrl = ctrl.ControlSystem(all_rules)
risk_sim = ctrl.ControlSystemSimulation(risk_ctrl)
print("Fuzzy Control System and Simulation object created.")

Fuzzy Control System and Simulation object created.


In [ ]:
🔷 D5 — Test the System on Sample Inputs

In [11]:
print("\n--- Testing Samples ---")

# --- Case 1: Optimal Health (Expected: VLOW) ---
# Inputs: BMI=Healthy, Age=Young, HighBP=Normal, Smoker=No, Activity=Active
risk_sim.input['BMI'] = 0.35
risk_sim.input['Age'] = 0.1
risk_sim.input['HighBP'] = 0.0
risk_sim.input['Smoker'] = 0.0
risk_sim.input['PhysActivity'] = 1.0
risk_sim.compute()
print(f"Case 1 (Optimal Health - VLOW Expected) Output: {risk_sim.output['Risk']:.4f}")

# --- Case 2: Overweight Smoker (Expected: LOW) ---
# Inputs: BMI=Over, Age=Middle, HighBP=Normal, Smoker=Yes, Activity=Active
# Targets Rule 4 & 5
risk_sim.input['BMI'] = 0.5
risk_sim.input['Age'] = 0.5
risk_sim.input['HighBP'] = 0.0
risk_sim.input['Smoker'] = 1.0
risk_sim.input['PhysActivity'] = 1.0
risk_sim.compute()
print(f"Case 2 (Overweight Smoker - LOW Expected) Output: {risk_sim.output['Risk']:.4f}")

# --- Case 3: Obese but Young and Active (Original Test Case - Expected: MED) ---
# Inputs: BMI=Obese, Age=Young, HighBP=Normal, Smoker=No, Activity=Active
# Targets Rule 7
risk_sim.input['BMI'] = 0.8
risk_sim.input['Age'] = 0.2
risk_sim.input['HighBP'] = 0.0
risk_sim.input['Smoker'] = 0.0
risk_sim.input['PhysActivity'] = 1.0
risk_sim.compute()
print(f"Case 3 (Obese, Young/Active - MED Expected) Output: {risk_sim.output['Risk']:.4f}")

# --- Case 4: High BP and Inactive, but Healthy Weight (Expected: HIGH) ---
# Inputs: BMI=Healthy, Age=Old, HighBP=High, Smoker=No, Activity=Inactive
# Targets Rule 17 (Old, High BP)
risk_sim.input['BMI'] = 0.35
risk_sim.input['Age'] = 0.8
risk_sim.input['HighBP'] = 1.0
risk_sim.input['Smoker'] = 0.0
risk_sim.input['PhysActivity'] = 0.0
risk_sim.compute()
print(f"Case 4 (High BP, Old, Inactive - HIGH Expected) Output: {risk_sim.output['Risk']:.4f}")

# --- Case 5: Triple Threat (Expected: VHIGH) ---
# Inputs: BMI=Obese, Age=Old, HighBP=High, Smoker=Yes, Activity=Inactive
# Targets Rule 18 & 19
risk_sim.input['BMI'] = 0.95
risk_sim.input['Age'] = 0.95
risk_sim.input['HighBP'] = 1.0
risk_sim.input['Smoker'] = 1.0
risk_sim.input['PhysActivity'] = 0.0
risk_sim.compute()
print(f"Case 5 (Triple Threat - VHIGH Expected) Output: {risk_sim.output['Risk']:.4f}")


--- Testing Samples ---
Case 1 (Optimal Health - VLOW Expected) Output: 0.0700
Case 2 (Overweight Smoker - LOW Expected) Output: 0.2500
Case 3 (Obese, Young/Active - MED Expected) Output: 0.5000
Case 4 (High BP, Old, Inactive - HIGH Expected) Output: 0.6705
Case 5 (Triple Threat - VHIGH Expected) Output: 0.8214


In [ ]:
🔷 D6 & D7 — Evaluation Loop and Metrics